# Food Delivery Data Analysis
This notebook demonstrates how to load, merge, and analyze food delivery data from multiple sources (CSV, JSON, SQL).


In [ ]:
import pandas as pd
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns


## Step 1: Load Data
We start by loading the `orders.csv` and `users.json` files.


In [ ]:
# Load Orders
orders_df = pd.read_csv('orders.csv')
print('Orders shape:', orders_df.shape)

# Load Users
users_df = pd.read_json('users.json')
print('Users shape:', users_df.shape)


## Step 2: Parse SQL Data
Since the restaurant data is in a SQL dump file (`restaurants.sql`), we need to parse the `INSERT` statements to extract the data.


In [ ]:
restaurants_data = []
with open('restaurants.sql', 'r') as file:
    sql_content = file.read()
    # Regex to capture values inside INSERT INTO restaurants VALUES (...);
    pattern = re.compile(r"INSERT INTO restaurants VALUES \((.*?)\);", re.IGNORECASE)
    matches = pattern.findall(sql_content)
    
    for match in matches:
        try:
            # Eval the string to a tuple
            row = eval(match)
            restaurants_data.append(row)
        except Exception as e:
            print(f'Error parsing row: {e}')

restaurants_df = pd.DataFrame(restaurants_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
print('Restaurants shape:', restaurants_df.shape)
restaurants_df.head()


## Step 3: Merge Datasets
We will merge the datasets using `user_id` and `restaurant_id` as keys.


In [ ]:
# Merge Orders with Users
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Merge with Restaurants
# Note: 'restaurant_name' exists in orders (as descriptive name) and restaurants table.
# We will suffix them to distinguish.
final_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_info'))

# Rename columns for clarity
final_df.rename(columns={
    'name': 'user_name',
    'restaurant_name_info': 'restaurant_name_std',
    'restaurant_name_order': 'restaurant_name_desc'
}, inplace=True)

# Convert order_date to datetime
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')

print('Final Dataset Shape:', final_df.shape)
final_df.head()


## Step 4: Data Analysis
Now we can answer specific business questions.


In [ ]:
# Q: Total Revenue by City for Gold Members
gold_revenue = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)
print('Gold Member Revenue by City:')
print(gold_revenue)


In [ ]:
# Q: Average Order Value by Cuisine
aov_cuisine = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print('Average Order Value by Cuisine:')
print(aov_cuisine)


In [ ]:
# Q: Revenue by Restaurant Rating Range
def bin_rating(r):
    if 3.0 <= r <= 3.5: return '3.0 – 3.5'
    elif 3.6 <= r <= 4.0: return '3.6 – 4.0'
    elif 4.1 <= r <= 4.5: return '4.1 – 4.5'
    elif 4.6 <= r <= 5.0: return '4.6 – 5.0'
    return 'Other'

final_df['rating_range'] = final_df['rating'].apply(bin_rating)
rating_revenue = final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)
print('Revenue by Rating Range:')
print(rating_revenue)


## Step 5: Export Data
Finally, we save the merged dataset to a CSV file.


In [ ]:
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print('File saved successfully!')
